# Анализ лояльности пользователей Яндекс Афиши

## Этапы выполнения проекта

### 1. Загрузка данных и их предобработка

---

**Задача 1.1:** Напишите SQL-запрос, выгружающий в датафрейм pandas необходимые данные. Используйте следующие параметры для подключения к базе данных `data-analyst-afisha`:

- **Хост** — `rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net`
- **База данных** — `data-analyst-afisha`
- **Порт** — `6432`
- **Аутентификация** — `Database Native`
- **Пользователь** — `praktikum_student`
- **Пароль** — `Sdf4$2;d-d30pp`

Для выгрузки используйте запрос из предыдущего урока и библиотеку SQLAlchemy.

Выгрузка из базы данных SQL должна позволить собрать следующие данные:

- `user_id` — уникальный идентификатор пользователя, совершившего заказ;
- `device_type_canonical` — тип устройства, с которого был оформлен заказ (`mobile` — мобильные устройства, `desktop` — стационарные);
- `order_id` — уникальный идентификатор заказа;
- `order_dt` — дата создания заказа (используйте данные `created_dt_msk`);
- `order_ts` — дата и время создания заказа (используйте данные `created_ts_msk`);
- `currency_code` — валюта оплаты;
- `revenue` — выручка от заказа;
- `tickets_count` — количество купленных билетов;
- `days_since_prev` — количество дней от предыдущей покупки пользователя, для пользователей с одной покупкой — значение пропущено;
- `event_id` — уникальный идентификатор мероприятия;
- `service_name` — название билетного оператора;
- `event_type_main` — основной тип мероприятия (театральная постановка, концерт и так далее);
- `region_name` — название региона, в котором прошло мероприятие;
- `city_name` — название города, в котором прошло мероприятие.

---


In [ ]:
#????????? ? ????????? pandas ??????????? ??????
from phik import phik_matrix
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
from urllib.parse import quote_plus
import os

load_dotenv()

db_config = {
    'user': os.getenv('DB_USER'),  # ??? ????????????
    'pwd': os.getenv('DB_PASSWORD'),  # ??????
    'host': os.getenv('DB_HOST'),
    'port': os.getenv('DB_PORT'),  # ???? ???????????
    'db': os.getenv('DB_NAME')  # ???????? ???? ??????
}

pwd = quote_plus(db_config['pwd'] or '')
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
    db_config['user'],
    pwd,
    db_config['host'],
    db_config['port'],
    db_config['db'],
)
engine = create_engine(connection_string)

query = '''
WITH set_config_precode AS (
  SELECT set_config('synchronize_seqscans', 'off', true)
)
SELECT
    -- ?????? ? ??????
    p.user_id,
    p.device_type_canonical,
    p.order_id,
    p.created_dt_msk AS order_dt,
    p.created_ts_msk AS order_ts,
    p.currency_code,
    p.revenue,
    p.tickets_count,
    (
      p.created_dt_msk::date
      - LAG(p.created_dt_msk::date) OVER (
          PARTITION BY p.user_id
          ORDER BY p.created_dt_msk::date
        )
    ) AS days_since_prev,

    -- ?????? ? ??????????? ? ????? ??????????
    p.event_id,
    e.event_name_code AS event_name,
    e.event_type_main,
    p.service_name,
    r.region_name,
    c.city_name
FROM afisha.purchases AS p
INNER JOIN afisha.events  AS e ON e.event_id = p.event_id
INNER JOIN afisha.city    AS c ON c.city_id = e.city_id
INNER JOIN afisha.regions AS r ON r.region_id = c.region_id
WHERE p.device_type_canonical IN ('mobile', 'desktop')
  AND e.event_type_main != '?????'
ORDER BY p.user_id;
'''
df = pd.read_sql_query(query, con=engine)


In [ ]:
#выводим первые 10 строк датафрейма для ознакомления
print(df.head(10))

---

**Задача 1.2:** Изучите общую информацию о выгруженных данных. Оцените корректность выгрузки и объём полученных данных.

Предположите, какие шаги необходимо сделать на стадии предобработки данных — например, скорректировать типы данных.

Зафиксируйте основную информацию о данных в кратком промежуточном выводе.

---

In [ ]:
#изучим общую информацию о данных и ее корректность
print(df.info())

выгружены данные. Кол-во строк составляет - 290611, все значения в датафрейме - ненулевые, пропуски присутствуют только в столбце days_since_prev

---

###  2. Предобработка данных

Выполните все стандартные действия по предобработке данных:

---

**Задача 2.1:** Данные о выручке сервиса представлены в российских рублях и казахстанских тенге. Приведите выручку к единой валюте — российскому рублю.

Для этого используйте датасет с информацией о курсе казахстанского тенге по отношению к российскому рублю за 2024 год — `final_tickets_tenge_df.csv`. Его можно загрузить по пути `https://code.s3.yandex.net/datasets/final_tickets_tenge_df.csv')`

Значения в рублях представлено для 100 тенге.

Результаты преобразования сохраните в новый столбец `revenue_rub`.

---


In [ ]:
#приведем данные к нужным типам и добавим расчет выручки в рублях
rate = pd.read_csv("https://code.s3.yandex.net/datasets/final_tickets_tenge_df.csv")
rate["data"] = pd.to_datetime(rate["data"]).dt.date

df["order_dt"] = pd.to_datetime(df["order_dt"]).dt.date

df = df.merge(
    rate[["curs", "cdx"]],
    how="left",
    left_on = df["order_dt"],
    right_on = rate["data"]
)

df["revenue_rub"] = np.where(
    df["cdx"].str.lower().eq("kzt"),
    df["revenue"] * df["curs"] / 100,
    df["revenue"]
)

df = df.drop(columns=["key_0", "cdx", "curs"])
print(df.columns)
print(df.head())

провели расчет выручки из казахстанских тенге в российские рубли

---

**Задача 2.2:**

- Проверьте данные на пропущенные значения. Если выгрузка из SQL была успешной, то пропуски должны быть только в столбце `days_since_prev`.
- Преобразуйте типы данных в некоторых столбцах, если это необходимо. Обратите внимание на данные с датой и временем, а также на числовые данные, размерность которых можно сократить.
- Изучите значения в ключевых столбцах. Обработайте ошибки, если обнаружите их.
    - Проверьте, какие категории указаны в столбцах с номинальными данными. Есть ли среди категорий такие, что обозначают пропуски в данных или отсутствие информации? Проведите нормализацию данных, если это необходимо.
    - Проверьте распределение численных данных и наличие в них выбросов. Для этого используйте статистические показатели, гистограммы распределения значений или диаграммы размаха.
        
        Важные показатели в рамках поставленной задачи — это выручка с заказа (`revenue_rub`) и количество билетов в заказе (`tickets_count`), поэтому в первую очередь проверьте данные в этих столбцах.
        
        Если обнаружите выбросы в поле `revenue_rub`, то отфильтруйте значения по 99 перцентилю.

После предобработки проверьте, были ли отфильтрованы данные. Если были, то оцените, в каком объёме. Сформулируйте промежуточный вывод, зафиксировав основные действия и описания новых столбцов.

---

In [ ]:
#проверка столбцов на пропуски
print(df.isna().sum())

In [ ]:
#преобразуем типы данных в числовых столбцах
df[['order_id', 'tickets_count', 'event_id']] = df[['order_id', 'tickets_count', 'event_id']].astype('int8')
df[['revenue', 'days_since_prev']] = df[['revenue', 'days_since_prev']].astype('float16')

In [ ]:
#проверка на дубликаты
print(df.duplicated().sum())

In [ ]:
#проверим распредление данных и наличие в них выбросов
import matplotlib.pyplot as plt
df[['revenue_rub', 'tickets_count']].describe()

In [ ]:
#визуализируем распределения для столбцов через boxplot
p99 = df["revenue_rub"].quantile(0.99)
df = df[df["revenue_rub"] <= p99]
print(df['revenue_rub'].describe())
print(df['tickets_count'].describe())

plt.figure(figsize=(12, 6))
plt.boxplot(df[['revenue_rub', 'tickets_count']])
plt.title('Boxplot для рублевой выручки и количества билетов')
plt.ylabel('Значения')
plt.show()

провели анализ распредления в ключевых столбцах через боксплот для наглядной демонстрации распредления значений и выбросов. Обе категории имеют высокую выраженность выбросов, правую ассиметрию, высокие показатели стандартного отклонения в процентном измерении. Фильтрация по 99 перцентилю позволила избавиться от 8633 значений выбросов, что сильно упростило статистический аналз.

---

### 3. Создание профиля пользователя

В будущем отдел маркетинга планирует создать модель для прогнозирования возврата пользователей. Поэтому сейчас они просят вас построить агрегированные признаки, описывающие поведение и профиль каждого пользователя.

---

**Задача 3.1.** Постройте профиль пользователя — для каждого пользователя найдите:

- дату первого и последнего заказа;
- устройство, с которого был сделан первый заказ;
- регион, в котором был сделан первый заказ;
- билетного партнёра, к которому обращались при первом заказе;
- жанр первого посещённого мероприятия (используйте поле `event_type_main`);
- общее количество заказов;
- средняя выручка с одного заказа в рублях;
- среднее количество билетов в заказе;
- среднее время между заказами.

После этого добавьте два бинарных признака:

- `is_two` — совершил ли пользователь 2 и более заказа;
- `is_five` — совершил ли пользователь 5 и более заказов.

**Рекомендация:** перед тем как строить профиль, отсортируйте данные по времени совершения заказа.

---


In [ ]:
#сортируем датафрейм по времени заказа и с группировкой по user_id для дальнейшего анализа
df1 = df.sort_values(by="order_ts", ascending=True)

In [ ]:
#дата первого заказа для каждого пользователя
df1['first_order'] = df1.groupby("user_id")["order_ts"].transform('min')
print(df1['first_order'].head(10))

In [ ]:
#дата последнего заказа для каждого пользователя
df1['last_order'] = df1.groupby("user_id")["order_ts"].transform('max')
print(df1['last_order'].head(10))

In [ ]:
#устройство, с которого был сделан первый заказ
df1['first_device'] = df1.groupby("user_id")["device_type_canonical"].transform('min')
print(df1['first_device'].head(10))

In [ ]:
#регион, в котором был сделан первый заказ
df1['first_region'] = df1.groupby("user_id")["region_name"].transform('min')
print(df1['first_region'].head(10))

In [ ]:
#билетный оператор при первом заказе
df1['first_ticket_operator'] = df1.groupby("user_id")["service_name"].transform('min')
print(df1['first_ticket_operator'].head(10))

In [ ]:
print(df1['user_id'].nunique())

In [ ]:
#жанр первого посещенного мероприятия
df1['first_event_type'] = df1.groupby("user_id")["event_type_main"].transform('min')
print(df1['first_event_type'].head(10))

In [ ]:
#общее колво заказов для каждого пользователя
df1['total_orders'] = df1.groupby("user_id")["order_id"].transform('count')
print(df1['total_orders'].head(10))

In [ ]:
#средняя выручка на пользователя в рублях
df1['avg_revenue_per_user'] = df1.groupby("user_id")["revenue_rub"].transform('mean')
print(df1['avg_revenue_per_user'].head(10))

In [ ]:
#среднее колво билетов на пользователя
df1['avg_tickets_per_user'] = df1.groupby("user_id")["tickets_count"].transform('mean')
print(df1['avg_tickets_per_user'].head(10))

In [ ]:
#среднее время между заказами для каждого пользователя
df1['avg_time_between_orders'] = df1.groupby("user_id")["days_since_prev"].transform('mean')
print(df1['avg_time_between_orders'].head(10))

In [ ]:
df1 = df1.drop(columns=[
    "device_type_canonical",
    "order_dt",
    "order_ts",
    "currency_code",
    "revenue",
    "tickets_count",
    "event_id",
    "service_name",
    "event_type_main",
    "region_name",
    "city_name", 
    'event_name',
    'revenue_rub',
    'days_since_prev'
])
print(df1.head(10))

---

**Задача 3.2.** Прежде чем проводить исследовательский анализ данных и делать выводы, важно понять, с какими данными вы работаете: насколько они репрезентативны и нет ли в них аномалий.

Используя данные о профилях пользователей, рассчитайте:

- общее число пользователей в выборке;
- среднюю выручку с одного заказа;
- долю пользователей, совершивших 2 и более заказа;
- долю пользователей, совершивших 5 и более заказов.

Также изучите статистические показатели:

- по общему числу заказов;
- по среднему числу билетов в заказе;
- по среднему количеству дней между покупками.

По результатам оцените данные: достаточно ли их по объёму, есть ли аномальные значения в данных о количестве заказов и среднем количестве билетов?

Если вы найдёте аномальные значения, опишите их и примите обоснованное решение о том, как с ними поступить:

- Оставить и учитывать их при анализе?
- Отфильтровать данные по какому-то значению, например, по 95-му или 99-му перцентилю?

Если вы проведёте фильтрацию, то вычислите объём отфильтрованных данных и выведите статистические показатели по обновлённому датасету.

In [ ]:
#колво уникальных пользователей в дф
print(df1['user_id'].nunique())

In [ ]:
#средняя выручка с одного заказа
print(df1['avg_revenue_per_user'].mean())

In [ ]:
#доля пользователей, совершивших более 2 заказов
print((df1['total_orders'] > 2).sum() / df1['user_id'].nunique())

#доля пользователей, совершивших более 5 заказов
print((df1['total_orders'] > 5).sum() / df1['user_id'].nunique())

In [ ]:
#статистика через боксплот по общему числу заказов, по среднему числу билетов в заказе, по среднему колву дней между заказами
plt.figure(figsize=(8, 6))
p99 = df1["total_orders"]
plt.boxplot(df1['total_orders'])
plt.title('Boxplot для общего числа заказов на пользователя')
plt.show()

plt.figure(figsize=(8, 6))
plt.boxplot(df1['avg_tickets_per_user'])
plt.title('Boxplot для среднего числа билетов в заказе на пользователя')
plt.show()

plt.figure(figsize=(10, 6))
plt.hist(df1['avg_time_between_orders'], bins=100)
plt.title('Гистограмма для среднего количества времени между заказами на пользователя')
plt.show()

In [ ]:
print(df1['avg_time_between_orders'].nunique())

Заметны выбросы для общего колва заказов и среднего числа билетов в заказе. Для этих категорий введем ограничение в 95 перцентилей. Данных по среднему времени между заказами недостаточно для анализа, для них вводить ограничения не будем 

In [ ]:
#исключение выбросов выше 95 перцентиля 
plt.figure(figsize=(8, 6))
p95 = df1["total_orders"].quantile(0.95)
plt.boxplot(df1.loc[df1["total_orders"] <= p95, "total_orders"])
plt.title("Boxplot для общего числа заказов на пользователя (до 95 перцентиля)")
plt.show()

plt.figure(figsize=(8, 6))
p95 = df1["avg_tickets_per_user"].quantile(0.95)
plt.boxplot(df1.loc[df1["avg_tickets_per_user"] <= p95, "avg_tickets_per_user"])
plt.title("Boxplot для среднего числа билетов на пользователя (до 95 перцентиля)")
plt.show()

plt.figure(figsize=(8, 6))
p95 = df1["avg_time_between_orders"].quantile(0.95)
plt.hist(df1.loc[df1["avg_time_between_orders"] <= p95, "avg_time_between_orders"], bins=50)
plt.title("Гистограмма для среднего количества времени между заказами на пользователя (до 95 перцентиля)")
plt.xlabel("Среднее количество дней между заказами")
plt.ylabel("Частота")
plt.show()

In [ ]:
#вывод статистических показателей по обновленным столбцам
print((df1.loc[df1["avg_tickets_per_user"] <= p95, "avg_tickets_per_user"]).describe())
print('\n')
print((df1.loc[df1["total_orders"] <= p95, "total_orders"]).describe())
print('\n')
print((df1.loc[df1["avg_time_between_orders"] <= p95, "avg_time_between_orders"]).describe())

In [ ]:
#считаем объем отфильтрованных данных
cols = ["avg_tickets_per_user", "total_orders", "avg_time_between_orders"]

for c in cols:
    p95 = df1[c].quantile(0.95)
    filtered = df1.shape[0] - df1[df1[c] <= p95].shape[0]
    print(c,', ' "отфильтровано:", filtered)

---

### 4. Исследовательский анализ данных

Следующий этап — исследование признаков, влияющих на возврат пользователей, то есть на совершение повторного заказа. Для этого используйте профили пользователей.



#### 4.1. Исследование признаков первого заказа и их связи с возвращением на платформу

Исследуйте признаки, описывающие первый заказ пользователя, и выясните, влияют ли они на вероятность возвращения пользователя.

---

**Задача 4.1.1.** Изучите распределение пользователей по признакам.

- Сгруппируйте пользователей:
    - по типу их первого мероприятия;
    - по типу устройства, с которого совершена первая покупка;
    - по региону проведения мероприятия из первого заказа;
    - по билетному оператору, продавшему билеты на первый заказ.
- Подсчитайте общее количество пользователей в каждом сегменте и их долю в разрезе каждого признака. Сегмент — это группа пользователей, объединённых определённым признаком, то есть объединённые принадлежностью к категории. Например, все клиенты, сделавшие первый заказ с мобильного телефона, — это сегмент.
- Ответьте на вопрос: равномерно ли распределены пользователи по сегментам или есть выраженные «точки входа» — сегменты с наибольшим числом пользователей?

---


In [ ]:
#по типу первого мероприятия
print(df1.groupby('first_event_type').size() / df1.shape[0])
print('\n')
print(df1['first_event_type'].value_counts())

In [ ]:
#по типу устройства с которого совершена первая покупка
print(df1.groupby('first_device').size() / df1.shape[0])
print('\n')
print(df1['first_device'].value_counts())

In [ ]:
#по региону проведения мероприятия
print(df1.groupby('first_region').size() / df1.shape[0])
print('\n')
print(df1['first_region'].value_counts())

In [ ]:
#по билетному оператору при первом заказе
print(df1.groupby('first_ticket_operator').size() / df1.shape[0])
print('\n')
print(df1['first_ticket_operator'].value_counts())

Ответ на вопрос

---

**Задача 4.1.2.** Проанализируйте возвраты пользователей:

- Для каждого сегмента вычислите долю пользователей, совершивших два и более заказа.
- Визуализируйте результат подходящим графиком. Если сегментов слишком много, то поместите на график только 10 сегментов с наибольшим количеством пользователей. Такое возможно с сегментами по региону и по билетному оператору.
- Ответьте на вопросы:
    - Какие сегменты пользователей чаще возвращаются на Яндекс Афишу?
    - Наблюдаются ли успешные «точки входа» — такие сегменты, в которых пользователи чаще совершают повторный заказ, чем в среднем по выборке?

При интерпретации результатов учитывайте размер сегментов: если в сегменте мало пользователей (например, десятки), то доли могут быть нестабильными и недостоверными, то есть показывать широкую вариацию значений.

---


In [ ]:
#доля совершивших 2 и более заказов для сегмента первого мероприятия
share = (
    df1.groupby("first_event_type")
       .apply(lambda g: (g["total_orders"] >= 2).mean())
)

plt.figure(figsize = (8,6))
plt.bar(share.index, share.values)
plt.xlabel("Тип первого мероприятия")
plt.ylabel("Доля пользователей с 2 и более заказами")
plt.show()

In [ ]:
#доля совершивших 2 и более заказов для сегмента типа устройства
share = (
    df1.groupby("first_device")
       .apply(lambda g: (g["total_orders"] >= 2).mean())
)

plt.figure(figsize = (8,6))
plt.bar(share.index, share.values)
plt.xlabel("Тип устройства")
plt.ylabel("Доля пользователей с 2 и более заказами")
plt.show()

In [ ]:
#доля совершивших 2 и более заказа для сегмента региона
share = (
    df1.groupby("first_region")
       .apply(lambda g: (g["total_orders"] >= 2).mean())
)

plt.figure(figsize = (8,6))
plt.bar(share.index, share.values)
plt.xlabel("Регион")
plt.ylabel("Доля пользователей с 2 и более заказами")
plt.xticks(rotation=45, ha="right")
plt.show()

print(share)

In [ ]:
#доля совершивших 2 и более заказа для сегмента билетного оператора
share = (
    df1.groupby("first_ticket_operator")
       .apply(lambda g: (g["total_orders"] >= 2).mean()).head(10)
)

plt.figure(figsize = (8,6))
plt.bar(share.index, share.values)
plt.xlabel("Билетный оператор")
plt.ylabel("Доля пользователей с 2 и более заказами")
plt.xticks(rotation=45, ha="right")
plt.show()

ВЫВОД

---

**Задача 4.1.3.** Опираясь на выводы из задач выше, проверьте продуктовые гипотезы:

- **Гипотеза 1.** Тип мероприятия влияет на вероятность возврата на Яндекс Афишу: пользователи, которые совершили первый заказ на спортивные мероприятия, совершают повторный заказ чаще, чем пользователи, оформившие свой первый заказ на концерты.
- **Гипотеза 2.** В регионах, где больше всего пользователей посещают мероприятия, выше доля повторных заказов, чем в менее активных регионах.

---

Гипотеза 1:
Гипотеза верна, что напрямую следует из выводов предыдущей задачи с группировкой по типу мероприятия и колвом заказов >= 2. пользователи совершившие первый заказ на спортивные мероприятия чаще совершают повторные заказы чем пользователи, сделавшие первый заказ на концерты. Результаты приведены на барплоте

In [ ]:
#Гипотеза 2
share = (
    df1.groupby("first_region")
       .apply(lambda g: (g["total_orders"] >= 2).mean()).head(10)
)

top_regs = df1['first_region'].value_counts().head(10)
print(list(set(share.index) & (set(top_regs.index))))

Из результатов кода ячейки выше следует, что в 70% случаев повторные заказы совершаются в регионах, входящих в топ-10 по доле посещения мероприятий следовательно гипотеза2 - верна

---

#### 4.2. Исследование поведения пользователей через показатели выручки и состава заказа

Изучите количественные характеристики заказов пользователей, чтобы узнать среднюю выручку сервиса с заказа и количество билетов, которое пользователи обычно покупают.

Эти метрики важны не только для оценки выручки, но и для оценки вовлечённости пользователей. Возможно, пользователи с более крупными и дорогими заказами более заинтересованы в сервисе и поэтому чаще возвращаются.

---

**Задача 4.2.1.** Проследите связь между средней выручкой сервиса с заказа и повторными заказами.

- Постройте сравнительные гистограммы распределения средней выручки с билета (`avg_revenue_rub`):
    - для пользователей, совершивших один заказ;
    - для вернувшихся пользователей, совершивших 2 и более заказа.
- Ответьте на вопросы:
    - В каких диапазонах средней выручки концентрируются пользователи из каждой группы?
    - Есть ли различия между группами?

Текст на сером фоне:
    
**Рекомендация:**

1. Используйте одинаковые интервалы (`bins`) и прозрачность (`alpha`), чтобы визуально сопоставить распределения.
2. Задайте параметру `density` значение `True`, чтобы сравнивать форму распределений, даже если число пользователей в группах отличается.

---


In [ ]:
# средняя выручка с билета для пользователя, совершившего 1 заказ
avg_revenue_rub1 = df1.loc[df1["total_orders"] == 1, "avg_revenue_per_user"]
print(avg_revenue_rub1.describe(), "\n")

plt.figure(figsize=(8, 6))
plt.hist(avg_revenue_rub1, bins=50, alpha=0.6, density=True)
plt.title("Распределение средней выручки с билета (1 заказ)")
plt.xlabel("выручка")
plt.ylabel("Плотность")
plt.show()

# средняя выручка с билета для пользователя, совершившего 2+ заказов
avg_revenue_rub2 = df1.loc[df1["total_orders"] >= 2, "avg_revenue_per_user"]
print(avg_revenue_rub2.describe(), "\n")

plt.figure(figsize=(8, 6))
plt.hist(avg_revenue_rub2, bins=50, alpha=0.6, density=True)
plt.title("Распределение средней выручки с билета (2+ заказа)")
plt.xlabel("выручка")
plt.ylabel("Плотность")
plt.show()

Ответы на вопросы:
Пользователи, совершившие 1 заказ имеют медиану по заказу равную 74, пользователи совершившие 2+ заказа имеют медиану равную 101. Распределение пользователей с 2+ заказами ближе к нормальному, чем у пользователей с 1 заказом. Выбросы по стоимости одного заказа встречаются чаще у пользователей, совершивших 1 заказ. 
Различие между группами заключается в средней стоиомсти заказа в рублях: пользователи с 2+ заказами имеют большую медиану по заказу

---

**Задача 4.2.2.** Сравните распределение по средней выручке с заказа в двух группах пользователей:

- совершившие 2–4 заказа;
- совершившие 5 и более заказов.

Ответьте на вопрос: есть ли различия по значению средней выручки с заказа между пользователями этих двух групп?

---


In [ ]:
#сравнение распределения по средней выручке для категорицй пользователей совершивших 2-4 заказа и более 5 заказов
avg_revenue_24 = df1.loc[df1.total_orders.between(2,4), "avg_revenue_per_user"]
avg_revenue_5 = df1.loc[df1.total_orders >= 5, "avg_revenue_per_user"]
print(avg_revenue_24.median(), "\n")
print(avg_revenue_5.median(), "\n")
#визуализируем
plt.figure(figsize=(8, 6))
plt.hist(avg_revenue_24, bins=50, alpha=0.5, density=True, label="2-4 заказа")
plt.hist(avg_revenue_5, bins=50, alpha=0.5, density=True, label="5 и более заказов")
plt.title("Распределение средней выручки с билета")
plt.xlabel("выручка")
plt.ylabel("Плотность")
plt.legend()
plt.show()

гистограмма распредления по категориям пользователей представлена на визуализации выше. Средние значения стоимости заказа для двух сегментов 2-4 заказа и 5 и более заказов составляет 92 и 102 соответственно, что может указывать на взаимосвязь: количества заказов и средней стоиомсти одного заказа 

---

**Задача 4.2.3.** Проанализируйте влияние среднего количества билетов в заказе на вероятность повторной покупки.

- Изучите распределение пользователей по среднему количеству билетов в заказе (`avg_tickets_count`) и опишите основные наблюдения.
- Разделите пользователей на несколько сегментов по среднему количеству билетов в заказе:
    - от 1 до 2 билетов;
    - от 2 до 3 билетов;
    - от 3 до 5 билетов;
    - от 5 и более билетов.
- Для каждого сегмента подсчитайте общее число пользователей и долю пользователей, совершивших повторные заказы.
- Ответьте на вопросы:
    - Как распределены пользователи по сегментам — равномерно или сконцентрировано?
    - Есть ли сегменты с аномально высокой или низкой долей повторных покупок?

---

In [ ]:
#изучение распределения пользователей по среднему колву билетов в заказе
mask_rep = df1["total_orders"] >= 2
s = df1["avg_tickets_per_user"]

users_12 = df1.loc[mask_rep & s.between(1, 2, inclusive="both"), "user_id"].nunique()
users_23 = df1.loc[mask_rep & (s > 2) & (s <= 3), "user_id"].nunique()
users_35 = df1.loc[mask_rep & (s > 3) & (s <= 5), "user_id"].nunique()
users_5p = df1.loc[mask_rep & (s > 5), "user_id"].nunique()


labels = ["1-2", "2-3", "3-5", "5+"]
values = [users_12, users_23, users_35, users_5p]

plt.figure(figsize=(8, 5))
plt.bar(labels, values)
plt.title("Количество уникальных пользователей по сегментам заказов")
plt.xlabel("Сегмент по avg_tickets_per_user")
plt.ylabel("Количество пользователей (unique user_id)")
plt.show()

arr = []
for i in values:
    arr.append(i / df1["user_id"].nunique() * 100)
print(f'доля пользователей: {arr}')

На барплоте выше представлено распределение пользователей по сегментам заказов в количественном и относительном форматах. Распредление не равномерно: доминируют пользователи на интервале 2-3.

---

#### 4.3. Исследование временных характеристик первого заказа и их влияния на повторные покупки

Изучите временные параметры, связанные с первым заказом пользователей:

- день недели первой покупки;
- время с момента первой покупки — лайфтайм;
- средний интервал между покупками пользователей с повторными заказами.

---

**Задача 4.3.1.** Проанализируйте, как день недели, в которой была совершена первая покупка, влияет на поведение пользователей.

- По данным даты первого заказа выделите день недели.
- Для каждого дня недели подсчитайте общее число пользователей и долю пользователей, совершивших повторные заказы. Результаты визуализируйте.
- Ответьте на вопрос: влияет ли день недели, в которую совершена первая покупка, на вероятность возврата клиента?

---


In [ ]:
#Выделение дня недели из первого заказа пользователя
df1["first_order"] = pd.to_datetime(df1["first_order"])
df1["weekday"] = df1["first_order"].dt.day_name()

rep_orders = df1.loc[df1['total_orders'] >= 2].groupby('weekday')['user_id'].nunique().sort_values(ascending = False)
rep_orders_share = df1.loc[df1['total_orders'] >= 2].groupby('weekday')['user_id'].nunique() / df1['user_id'].nunique()
print(rep_orders_share, '\n')

labels = ["Saturday", "Friday", "Tuesday", "Wednesday", "Thursday", "Monday", "Sunday"]

plt.figure(figsize=(8, 6))
plt.bar(labels, rep_orders)
plt.title("Количество уникальных пользователей по дням недели")
plt.xlabel("День недели")
plt.ylabel("Количество уникальных пользователей")
plt.show()

колво пользователей с 2 и более заказов отражено на барплоте. Присутствует взаимосвязь возврата клиента и первого дня совершения заказа: например число вернувшихся пользователей совершивших заказ в субботу превышает число вернувшихся пользователей совершивших заказ в воскресенье примерно на 500 единиц, что следует из барплота.   

---

**Задача 4.3.2.** Изучите, как средний интервал между заказами влияет на удержание клиентов.

- Рассчитайте среднее время между заказами для двух групп пользователей:
    - совершившие 2–4 заказа;
    - совершившие 5 и более заказов.
- Исследуйте, как средний интервал между заказами влияет на вероятность повторного заказа, и сделайте выводы.

---


In [ ]:
#среднее время между заказами для пользователей по сегментам по колву заказов
users_24 = df1.loc[(df1["total_orders"] > 2) & (df1["total_orders"] <= 4)].groupby('user_id')['avg_time_between_orders'].mean()
users_5 = df1.loc[df1["total_orders"] >= 5].groupby('user_id')['avg_time_between_orders'].mean()
print(users_24.mean(), users_5.mean())

Результаты выше демонстрируют что группа пользователей совершившая 5 и более заказов имеет вдвое меньшее среднее время между заказами чем группа пользователей с 2-4 заказами. Из этого следует что меньшее время между заказами повышает вероятность повторного заказа

---

#### 4.4. Корреляционный анализ количества покупок и признаков пользователя

Изучите, какие характеристики первого заказа и профиля пользователя могут быть связаны с числом покупок. Для этого используйте универсальный коэффициент корреляции `phi_k`, который позволяет анализировать как числовые, так и категориальные признаки.

---

**Задача 4.4.1:** Проведите корреляционный анализ:
- Рассчитайте коэффициент корреляции `phi_k` между признаками профиля пользователя и числом заказов (`total_orders`). При необходимости используйте параметр `interval_cols` для определения интервальных данных.
- Проанализируйте полученные результаты. Если полученные значения будут близки к нулю, проверьте разброс данных в `total_orders`. Такое возможно, когда в данных преобладает одно значение: в таком случае корреляционный анализ может показать отсутствие связей. Чтобы этого избежать, выделите сегменты пользователей по полю `total_orders`, а затем повторите корреляционный анализ. Выделите такие сегменты:
    - 1 заказ;
    - от 2 до 4 заказов;
    - от 5 и выше.
- Визуализируйте результат корреляции с помощью тепловой карты.
- Ответьте на вопрос: какие признаки наиболее связаны с количеством заказов?

---

In [ ]:
import seaborn as sb

correlation_matrix = df1[['days_since_prev', 'first_device', 'first_region', 'first_ticket_operator', 
                          'first_event_type', 'total_orders', 'avg_revenue_per_user', 
                          'avg_tickets_per_user', 'avg_time_between_orders']].phik_matrix()

# Выводим результат
print('Корреляционная матрица с коэффициентом phi_k для переменной total_orders')
correlation_matrix.loc[correlation_matrix.index != 'total_orders'][['total_orders']].sort_values(by='total_orders', ascending=False)

In [ ]:
plt.figure(figsize=(2, 6))

# Сохраняем матрицу корреляции признака total_orders с другими признаками клиента
data_heatmap = correlation_matrix.loc[correlation_matrix.index != 'total_orders'][['total_orders']].sort_values(by='total_orders', ascending=False)
sb.heatmap(data_heatmap,
            annot=True,
            fmt='.2f', 
            cmap='coolwarm',
            linewidths=0.5, 
            cbar=False 
           )

# Добавляем заголовок и подпись по оси Х
plt.title('Тепловая карта коэффициента корреляции')
plt.xlabel('Колво заказов')

# Выводим график
plt.show()

На основании данных полученных в результате плоттинга тепловой карты можно заключить что наибольшую взаимосвязь с количеством заказов показывают столбцы с регионом и билетным оператором. Значения составляют соответственно 0.73 и 0.58

### 5. Общий вывод и рекомендации

В конце проекта напишите общий вывод и рекомендации: расскажите заказчику, на что нужно обратить внимание. В выводах кратко укажите:

- **Информацию о данных**, с которыми вы работали, и то, как они были подготовлены: например, расскажите о фильтрации данных, переводе тенге в рубли, фильтрации выбросов.
- **Основные результаты анализа.** Например, укажите:
    - Сколько пользователей в выборке? Как распределены пользователи по числу заказов? Какие ещё статистические показатели вы подсчитали важным во время изучения данных?
    - Какие признаки первого заказа связаны с возвратом пользователей?
    - Как связаны средняя выручка и количество билетов в заказе с вероятностью повторных покупок?
    - Какие временные характеристики влияют на удержание (день недели, интервалы между покупками)?
    - Какие характеристики первого заказа и профиля пользователя могут быть связаны с числом покупок согласно результатам корреляционного анализа?
- Дополните выводы информацией, которая покажется вам важной и интересной. Следите за общим объёмом выводов — они должны быть компактными и ёмкими.

В конце предложите заказчику рекомендации о том, как именно действовать в его ситуации. Например, укажите, на какие сегменты пользователей стоит обратить внимание в первую очередь, а какие нуждаются в дополнительных маркетинговых усилиях.

Подготовка и предобработка данных
Полученные данные содержали пропуски только в столбце days_since_prev, не содержали дубликатов и были приведены к меньшей размерности для экономии памяти. Был создан отдельный столбец с выручкой в российских рублях для корректной дальнейшей работы. Также во время предобработки была проведена фильтрация выбросов данных для упрощения дальнейшего статистического анализа. После этого перешли к созданию профиля пользователя


Количество уникальных пользователей в выборке составляет - 21767. 
Список наиболее весомых признаков связанных с возвратом пользователя на платформу: тип первого мероприятия, регион проведения мероприятия, билетный оператор при первом заказе, тип устройства при совершении заказа.
Результаты анализа показали, что выставки - тип мероприятия обеспечивший наибольшую долю повторных покупок, пользователи совершавшие заказы с компьютера - чаще совершают повторный заказ, Crazy_ticket, Show_ticket - билетные операторы обеспечившие наибольшую долю повторных покупок, Белоярская область и Берестовский округ - два наиболее популярных места совершения заказа, обеспечивших повторные продажи
Анализ взаимосвязи выручки и повторного заказа показал следующее: пользователи с 2+ заказами имеют большую медиану по заказу, чем пользователи с одним заказом. Взаимосвязь среднего количества билетов в заказе показывает что с наибольшей частотой повторную покупку совершают пользователи с 2-3 билетами в заказе
Наибольшее число пользователей совершивших 2 и более покупок совершили свой первый заказ в субботу и пятницу. Анализ влияния среднего интервала между заказами показал что пользователи с 5+ заказами имеют вдвое меньший интервал времени между заказами, чем пользователи с 2-4 заказами
Проведенный корреляционный анализ показал что наибольшей взаимосвязью с числом покупок обладают следующие признаки: регион совершения первого заказа и билетный оператор при первом заказе

Из проведенного корреляционного анализа следует, что регион мероприятия и билетный оператор - наиболее важные компоненты совокупного количества заказов пользователей следовательно Заказчику стоит сконцентрировать свои маркетинговые усилия на регионах проведения будущих мероприятий, а также рекламе у конкретных билетных операторов которые демонстрируют высокие показатели трафика пользователей

### 6. Финализация проекта и публикация в Git

Когда вы закончите анализировать данные, оформите проект, а затем опубликуйте его.

Выполните следующие действия:

1. Создайте файл `.gitignore`. Добавьте в него все временные и чувствительные файлы, которые не должны попасть в репозиторий.
2. Сформируйте файл `requirements.txt`. Зафиксируйте все библиотеки, которые вы использовали в проекте.
3. Вынести все чувствительные данные (параметры подключения к базе) в `.env`файл.
4. Проверьте, что проект запускается и воспроизводим.
5. Загрузите проект в публичный репозиторий — например, на GitHub. Убедитесь, что все нужные файлы находятся в репозитории, исключая те, что в `.gitignore`. Ссылка на репозиторий понадобится для отправки проекта на проверку. Вставьте её в шаблон проекта в тетрадке Jupyter Notebook перед отправкой проекта на ревью.

**Ссылка на проект: https://github.com/yuskonst/praktikum_yus**